In [1]:
from pathlib import Path
from pydicom import dcmread
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
from scipy.ndimage import gaussian_filter
from scipy.interpolate import splrep, splev
from scipy.interpolate import CubicSpline
from scipy.interpolate import interp1d, RegularGridInterpolator
import scipy.optimize as optimize
import collections
plt.rcParams['image.cmap'] = 'gray'

In [2]:
cardio_path = Path().resolve().parents[2] / "dane" / "KARDIO ZAMKNIETE"
data_path = cardio_path / "A001" / "DICOM" / "P1" / "E1" / "S1"

In [3]:
def convert_sequence_to_array(path: Path):
    return np.stack([np.flip(dcmread(file).pixel_array) for file in data_path.iterdir()], axis=0)

In [4]:
stacked = convert_sequence_to_array(data_path)

In [5]:
path = Path().resolve().parents[1] / "dane" / "KARDIO ZAMKNIETE" / "A001" / "DICOM" / "P1" / "E1" / "S1"

In [6]:
def get_data(data_path: Path, skip_index: np.ndarray = np.concatenate((np.arange(61, 67), np.arange(68, 78),
                                                                       np.arange(79, 89), np.arange(90, 100),
                                                                       np.arange(101, 111), np.arange(113, 123),
                                                                       np.arange(124, 132)), axis=0)) -> np.ndarray:
    """
    Return array with CT image data

    Parameters
    ----------
    skip_index
    data_path : Path
        Path to folder with data

    Returns
    -------
    `np.ndarray`

    """
    return np.stack(
        [np.flip(dcmread(file).pixel_array) for idx, file in enumerate(data_path.iterdir()) if idx not in skip_index],
        axis=0)

In [7]:
path_to_first_file = next(data_path.glob("*"))
dcm_file = dcmread(path_to_first_file)
print(f"Number of images: {stacked.shape[0]}")
print(f"Size of image: {stacked.shape[1]}x{stacked.shape[2]}")
print(f"Pixel size: {dcm_file.PixelSpacing} mm")
print(f"Reconstruction center: {dcm_file[(0x7005, 0x1007)].value}")
pixel_size_x = dcm_file.PixelSpacing[1]
pixel_size_y = dcm_file.PixelSpacing[0]
pixel_size = dcm_file.PixelSpacing[1]
print(pixel_size_x)

Number of images: 512
Size of image: 512x512
Pixel size: [0.402, 0.402] mm
Reconstruction center: [308, 216]
0.402


In [9]:
ids = []
for file in cardio_path.rglob("*"):
    if file.is_file() and file.name != 'DICOMDIR':
        dcm_file = dcmread(file)
        try:
            ids.append( dcm_file[ (0x0010, 0x1000)].value)
        except KeyError:
            ids.append(dcm_file[ (0x0010, 0x0030)].value)

In [13]:
counter = collections.Counter(ids)

In [14]:
counter

Counter({'3112192': 3506,
         '3119757': 2718,
         '2144676': 4385,
         '19360121': 3835,
         '19380721': 1583,
         '19471126': 4511,
         '1616263': 2667,
         '3114613': 2329,
         '19550528': 2898,
         '102209': 2327})

In [15]:
print(counter)

Counter({'19471126': 4511, '2144676': 4385, '19360121': 3835, '3112192': 3506, '19550528': 2898, '3119757': 2718, '1616263': 2667, '3114613': 2329, '102209': 2327, '19380721': 1583})


In [21]:
counter.values()

dict_values([3506, 2718, 4385, 3835, 1583, 4511, 2667, 2329, 2898, 2327])